In [99]:
import re
def get_indicators(description):
    indicators = re.findall('\[[^\]]+\]', description)
    return set(re.sub('[\[\]]', '', indicator) for indicator in indicators)

In [121]:
import json
# load data
with open('data/indicators.json') as f:
    data = json.load(f)
# parse data by Statement
d = defaultdict(set)
titles = {}
for entry in data:
    titles[entry['Indicator']] = entry['Title']
for entry in data:
#     if entry['Statement']=="Balance Sheet":
#     if entry['Statement']=="Cash Flow":
    if entry['Statement']=="Income Statement":
        entry['Indicators'] = get_indicators(entry['Description'])
        print()
        entry['Indicators'] = set(titles[i] + ' (' + i  + ')' for i in entry['Indicators'])
        entry['Indicators'].add(titles[entry['Indicator']] + ' (' + entry['Indicator'] + ')')
        for i in entry['Indicators']:
            d[i].update(entry['Indicators'])
#         if entry['Indicator'] not in g:
        print("{Title} ({Indicator}) : {Description}\n".format(**entry))



Consolidated Income (CONSOLINC) : The portion of profit or loss for the period, net of income taxes, which is attributable to the consolidated entity, before the deduction of [NETINCNCI].


Cost of Revenue (COR) : The aggregate cost of goods produced and sold and services rendered during the reporting period.


Dividends per Basic Common Share (DPS) : Aggregate dividends declared during the period for each split-adjusted share of common stock outstanding.


Earning Before Interest & Taxes (EBIT) (EBIT) : Earnings Before Interest and Tax is calculated by adding [TAXEXP] and [INTEXP] back to [NETINC].


Earning Before Interest & Taxes (USD) (EBITUSD) : [EBIT] in USD, converted by [FXUSD].


Earnings per Basic Share (EPS) : Earnings per share as calculated and reported by the company. Approximates to the amount of [NETINC] for the period per each [SHARESWA].


Earnings per Diluted Share (EPSDIL) : Earnings per diluted share as calculated and reported by the company. Approximates to the a

In [114]:
def build_graph(d, node, graph):
    used = set(graph.keys())
    used.add(node)
    children = d[node] - used
    if children:
        graph[node] = list(children)
    for child in children:
        graph = build_graph(d, child, graph)
    return graph

In [122]:
# Get hierarchy
from collections import defaultdict
g = defaultdict(set)
# d['TOP'] = set(['Net Cash Flow / Change in Cash & Cash Equivalents (NCF)'])
# d['TOP'] = set(['Total Assets (ASSETS)', 'Total Liabilities (LIABILITIES)'])
d['TOP'] = set(['Revenues (REVENUE)'])

g = build_graph(d, 'TOP', g)
print(json.dumps(g, indent=4))

{
    "TOP": [
        "Revenues (REVENUE)"
    ],
    "Revenues (REVENUE)": [
        "Cost of Revenue (COR)",
        "Foreign Currency to USD Exchange Rate (FXUSD)",
        "Revenues (USD) (REVENUEUSD)",
        "Gross Profit (GP)"
    ],
    "Cost of Revenue (COR)": [
        "Operating Expenses (OPEX)",
        "Research and Development Expense (RND)",
        "Selling, General and Administrative Expense (SGNA)",
        "Gross Profit (GP)"
    ],
    "Operating Expenses (OPEX)": [
        "Operating Income (OPINC)",
        "Gross Profit (GP)",
        "Research and Development Expense (RND)",
        "Selling, General and Administrative Expense (SGNA)",
        "Income Tax Expense (TAXEXP)",
        "Interest Expense (INTEXP)"
    ],
    "Operating Income (OPINC)": [
        "Interest Expense (INTEXP)",
        "Income Tax Expense (TAXEXP)",
        "Gross Profit (GP)"
    ],
    "Interest Expense (INTEXP)": [
        "Gross Profit (GP)"
    ],
    "Net Income (NETINC)": [
    

# Balance Sheet
Total Assets (ASSETS)
    + Cash and Equivalents (CASHNEQ)
    + Investments (INVESTMENTS)
        + Investments Current (INVESTMENTSC)
        + Investments Non-Current (INVESTMENTSNC)
    + Goodwill and Intangible Assets (INTANGIBLES)
    + Property, Plant & Equipment Net (PPNENET)
    + Tax Assets (TAXASSETS)
    + Trade and Non-Trade Receivables (RECEIVABLES)
Total Liabilities (LIABILITIES)
    + Total Debt (DEBT)
        + Debt Current (DEBTC)
        + Debt Non-Current (DEBTNC)
    + Deferred Revenue (DEFERREDREV)
    + Trade and Non-Trade Payables (PAYABLES)
    + Deposit Liabilities (DEPOSITS)
    + Tax Liabilities (TAXLIABILITIES)
Shareholders Equity (EQUITY)
    + Accumulated Other Comprehensive Income (ACCOCI)
    + Accumulated Retained Earnings (Deficit) (RETEARN)

Consolidated Income [CONSOLINC] : The portion of profit or loss for the period, net of income taxes, which is attributable to the consolidated entity, before the deduction of [NETINCNCI].

Cost of Revenue [COR] : 

Dividends per Basic Common Share [DPS] : 

Earning Before Interest & Taxes [EBIT]  :  [NETINC] + [TAXEXP] + [INTEXP] 

Earning Before Interest & Taxes (USD) [EBITUSD] : [EBIT] * [FXUSD]

Earnings per Basic Share [EPS] : [NETINC] / [SHARESWA]

Earnings per Diluted Share [EPSDIL] :  [NETINC] / [SHARESWADIL]

Earnings per Basic Share (USD) [EPSUSD] : [EPS] * [FXUSD]

Gross Profit [GP] :  [REVENUE] - [COR] 

Interest Expense [INTEXP] : 

Net Income [NETINC] :   [CONSOLINC] - [NETINCNCI]

Net Income Common Stock [NETINCCMN] : [NETINC] + [PREFDIVIS]

Net Income Common Stock (USD) [NETINCCMNUSD] : [NETINCCMN] * [FXUSD]

Net Income from Discontinued Operations [NETINCDIS] : 

Net Income to Non-Controlling Interests [NETINCNCI] :   [CONSOLINC] - [NETINC]

Operating Expenses [OPEX] : [SGNA] + [RND] - [COR]

Operating Income [OPINC] : [GP] - [OPEX]

Preferred Dividends Income Statement Impact [PREFDIVIS] :  [NETINCCMN] - [NETINC]

Revenues [REVENUE] : 

Revenues (USD) [REVENUEUSD] : [REVENUE] * [FXUSD].

Research and Development Expense [RND] : A component of [OPEX] 

Selling, General and Administrative Expense [SGNA] : A component of [OPEX] 

Weighted Average Shares [SHARESWA] : The weighted average number of shares or units issued and outstanding that are used by the company to calculate [EPS], determined based on the timing of issuance of shares or units in the period.

Weighted Average Shares Diluted [SHARESWADIL] : The weighted average number of shares or units issued and outstanding that are used by the company to calculate [EPSDIL], determined based on the timing of issuance of shares or units in the period.

Income Tax Expense [TAXEXP] : 

Revenues [REVENUE]
    - Cost of Revenue [COR]
Gross Profit [GP]
    - Operating Expenses [OPEX]
        + Selling, General and Administrative Expense [SGNA]
        + Research and Development Expense [RND]
        - Cost of Revenue [COR]
Operating Income [OPINC]
    + (Non Operating Income
Earning Before Interest & Taxes [EBIT]
    - Income Tax Expense [TAXEXP]
    - Interest Expense [INTEXP]
Net Income [NETINC]
        + Net Income to Non-Controlling Interests [NETINCNCI]
    = Consolidated Income [CONSOLINC]
        + Preferred Dividends Income Statement Impact [PREFDIVIS]
    = Net Income Common Stock [NETINCCMN]


Capital Expenditure (CAPEX) : A component of [NCFI] 

Depreciation, Amortization & Accretion (DEPAMOR) : 

Net Cash Flow / Change in Cash & Cash Equivalents (NCF) : Includes [NCFO] + [NCFI] + [NCFF] * [NCFX].

Net Cash Flow - Business Acquisitions and Disposals (NCFBUS) : A component of [NCFI] 

Issuance (Purchase) of Equity Shares (NCFCOMMON) : A component of [NCFF] 

Issuance (Repayment) of Debt Securities  (NCFDEBT) : A component of [NCFF] 

Payment of Dividends & Other Cash Distributions    (NCFDIV) : A component of [NCFF] 

Net Cash Flow from Financing (NCFF) : A component of [NCF] 

Net Cash Flow from Investing (NCFI) : A component of [NCF]. Principal components of investing cash flow are: capital (expenditure) disposal of equipment [CAPEX], business (acquisitions) disposition [NCFBUS] and investment (acquisition) disposal [NCFINV].

Net Cash Flow - Investment Acquisitions and Disposals (NCFINV) : A component of [NCFI] 

Net Cash Flow from Operations (NCFO) : A component of [NCF] 

Effect of Exchange Rate Changes on Cash  (NCFX) : A component of Net Cash Flow [NCF] 

Share Based Compensation (SBCOMP) : A component of [NCFO] 

Net Cash Flow / Change in Cash & Cash Equivalents (NCF)
    + "Net Cash Flow from Investing (NCFI)",
    + "Effect of Exchange Rate Changes on Cash  (NCFX)",
    + "Net Cash Flow from Financing (NCFF)",
    + "Net Cash Flow - Investment Acquisitions and Disposals (NCFINV)",
    + "Capital Expenditure (CAPEX)",
    + "Net Cash Flow - Business Acquisitions and Disposals (NCFBUS)",
    + "Net Cash Flow from Operations (NCFO)"
"Net Cash Flow from Investing (NCFI)": 
    + "Effect of Exchange Rate Changes on Cash  (NCFX)",
    + "Net Cash Flow from Financing (NCFF)",
    + "Capital Expenditure (CAPEX)",
    + "Net Cash Flow - Investment Acquisitions and Disposals (NCFINV)",
    + "Net Cash Flow - Business Acquisitions and Disposals (NCFBUS)",
    + "Net Cash Flow from Operations (NCFO)"
"Effect of Exchange Rate Changes on Cash  (NCFX)": 
    + "Net Cash Flow from Financing (NCFF)",
    + "Net Cash Flow from Operations (NCFO)"
"Net Cash Flow from Financing (NCFF)": 
    + "Payment of Dividends & Other Cash Distributions    (NCFDIV)",
    + "Issuance (Purchase) of Equity Shares (NCFCOMMON)",
    + "Issuance (Repayment) of Debt Securities  (NCFDEBT)"
"Net Cash Flow from Operations (NCFO)": 
    + "Share Based Compensation (SBCOMP)"
"Capital Expenditure (CAPEX)": 
    + "Net Cash Flow - Business Acquisitions and Disposals (NCFBUS)"
"Net Cash Flow - Investment Acquisitions and Disposals (NCFINV)": 
    + "Net Cash Flow - Business Acquisitions and Disposals (NCFBUS)"
